# Uso de transformers con las `AutoClasses` de Hugging Face
Las clases `AutoClasses` nos permiten cargar la configuración, tokenizado y modelo de una arquitectura transformer concreta para distintas tareas de texto.  
>AutoClasses are here to do this job for you so that you automatically retrieve the relevant model given the name/path to the pretrained weights/config/vocabulary.
>Instantiating one of AutoConfig, AutoModel, and AutoTokenizer will directly create a class of the relevant architecture

In [ ]:
#instalamos la librería
!pip install transformers

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

Definimos un modelo (`checkpoint`) de una arquitectura concreta a cargar. Los posible modelos están listados en https://huggingface.co/docs/transformers/v4.29.1/en/model_doc/auto#transformers.AutoConfig.from_pretrained  


In [ ]:
checkpoint = 'bert-base-cased'

Cargamos el tokenizador específico

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer

In [ ]:
input = tokenizer("I like the Transformers library")

In [ ]:
print(input)

Cargamos la configuración por defecto del modelo

In [ ]:
config = AutoConfig.from_pretrained(checkpoint)

config

In [ ]:
[attr for attr in dir(config) if not attr.startswith('__')]

Podemos cambiar algunos parámetros de la configuración

In [ ]:
config.output_hidden_states = True

Cargamos un modelo base (head-less)

In [ ]:
modelo = AutoModel.from_pretrained(checkpoint, config=config)

In [ ]:
modelo

Podemos ver el detalle de cada capa con su estructura PyTorch

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(modelo)

La entrada a los modelos transformers es el texto tokenizado con el vectorizador correspondiente.

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [ ]:
inputs

Los modelos genéricos devuelve la última capa de salida del ENCODER (`last_hidden_states`)

In [ ]:
output = modelo(**inputs)

In [ ]:
output.keys()

In [ ]:
output.last_hidden_state.shape

Adicionalmente el modelo BERT también devuelve un embedding de documento en la salida `pooler_output`.

>`pooler_output` contains a "representation" of each sequence in the batch. What it basically does is take the hidden representation of the `[CLS]` token of each sequence in the batch, and then run that through the BertPooler nn.Module. This consists of a linear layer followed by a Tanh activation function. The weights of this linear layer are already pretrained on the next sentence prediction task

In [ ]:
output.pooler_output.shape

También se puede acceder a todas las representaciones de las capas intermedias del ENCODER (si en la configuración el parámetro `'config.output_hidden_states'` está a `True`):

In [ ]:
len(output.hidden_states)

In [ ]:
output.hidden_states[0].shape

## Modelos para una tarea específica
También podemos cargar la arquitectura (HEAD) para una tarea del lenguaje determinada. Existen las siguientes tareas:  https://huggingface.co/docs/transformers/v4.29.1/en/model_doc/auto#natural-language-processing
### Clasificación de textos

In [ ]:
from transformers import AutoModelForSequenceClassification

modelo = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
modelo

In [ ]:
output = modelo(**inputs)

En este caso el modelo devuelve las probabilidades (sin normalizar) de la capa densa de salida (nº de salidas igual al nº de clases definidas)

In [ ]:
output.keys()

In [ ]:
output.logits.shape

In [ ]:
output.logits

### Clasificación de tokens
P. ej. *Name Entity Recognition*, NER

In [ ]:
from transformers import AutoModelForTokenClassification

id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

modelo = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=13, id2label=id2label, label2id=label2id)

In [ ]:
modelo

In [ ]:
inputs = tokenizer(["I like icecream", "I do not like brocolli"], padding=True, return_tensors="pt")

In [ ]:
inputs

### Ejercicio
Calcula la salida del modelo sobre los documentos de entrada anteriores e interpreta el tamaño del tensor de salida.

In [ ]:
#Solución


## Modelos ajustados a una tarea
Podemos hacer uso de modelos ya ajustados (*fine-tuned*) a una tarea con un dataset específico para hacer inferencia en la misma tarea.
Para eso, tenemos que cargar el modelo elegido y pasarle como entrada el texto tokenizado con su vectorizador correspondiente

### Uso de los modelos ajustados en inferencia
Para usar estos modelos en nuestro flujo de trabajo (p. ej. como un modelo de `tensorflow.keras`) lo necesitamos cargar junto a su función de tokenizado específica.  
Por ejemplo, para un modelo de análisis de sentimientos:

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AutoConfig

nombre_modelo = "distilbert-base-uncased-finetuned-sst-2-english"
config = AutoConfig.from_pretrained(nombre_modelo)
config.output_hidden_states = True
tf_model = TFAutoModelForSequenceClassification.from_pretrained(nombre_modelo, config=config)
tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)


In [ ]:
tf_model.summary()

Para usar el modelo, primero convertimos la entrada en tokens

In [ ]:
docs = ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."]

tf_batch = tokenizer(
    docs,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf"
)

`tf_batch` genera un diccionario con `'inputs_ids'` y `'attention_mask'` para cada texto de entrada.

In [ ]:
for key, value in tf_batch.items():
    print(f"{key}: {value.numpy().tolist()}")

In [ ]:

print(tokenizer.convert_ids_to_tokens(tf_batch['input_ids'][0]))

Aplicamos el modelo, que devuelve los `logits` de la última capa y las salidas de cada capa intermedia (*embeddings*)

In [ ]:
tf_outputs = tf_model(tf_batch)
tf_outputs.keys()

In [ ]:
len(tf_outputs.hidden_states) #Nº de capas internas del transformer (embedding + 6 capas atención)

In [ ]:
tf_outputs.hidden_states[0].shape #embeddings de salida de cada capa (nª muestras, nº tokens, nº dimensiones)

In [ ]:
tf_outputs.logits #salida del modelo

Aplicamos la función de activación Softmax para obtener las probabilidades normalizadas de cada clase (negativo, positivo) a partir de los logits

In [ ]:
import tensorflow as tf
predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
print(predictions)

In [ ]:
import numpy as np

pred_class = np.argmax(predictions, axis=1)
pred_class

In [ ]:
[tf_model.config.id2label[c] for c in pred_class]

### Inferencia con `PyTorch`
También podemos cargar los modelos en `PyTorch`

In [ ]:
from transformers import AutoModelForSequenceClassification

config = AutoConfig.from_pretrained(nombre_modelo)
config.output_hidden_states = True
model = AutoModelForSequenceClassification.from_pretrained(nombre_modelo, config=config)


In [ ]:
model

In [ ]:
batch = tokenizer(
    docs,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [ ]:
batch.keys()

In [ ]:
batch.input_ids #ahora los arrays son tensores de pyTorch

In [ ]:
outputs = model(**batch)

In [ ]:
outputs.keys()

In [ ]:
outputs.logits

Convertimos las probabilidades *logits* a probabilidades normalizadas

In [ ]:
outputs.logits.softmax(dim=-1).tolist()

In [ ]:
outputs.logits.softmax(dim=-1).argmax(dim=-1)